In [1]:
from simple_coin_machine import SimpleCoinMachine
device = SimpleCoinMachine()

DEBUG:root:Initialised a coin machine, please change states by on_event() function call
DEBUG:root:Entered LockedState


In [10]:
device.on_event('pass')

This event pass is not supported in this state
Supported events are ['coin', 'failed']


In [3]:
device.on_event('pin_entered')

This event pin_entered is not supported in this state
Supported events are ['coin', 'failed']


In [18]:
device.on_event('coin')

current collected_amount is 0
please enter the amount of coins5
Still need 15 coins to unlock


In [12]:
device.on_event('coin')

current collected_amount is 16
please enter the amount of coins4


DEBUG:root:Entered UnlockedState


In [16]:
print (device.state)
device.on_event('failed')

DEBUG:root:Entered BrokenState


LockedState
Machine is Out of order


In [17]:
device.on_event('fixed')

DEBUG:root:Entered LockedState


In [8]:
device.on_event('coin')

This event coin is not supported in this state
Supported events are ['fixed']


In [9]:
device.on_event('fixed')

DEBUG:root:Entered LockedState
